<a href="https://colab.research.google.com/github/jagrutsharma/hack-2022/blob/main/Hackathon_2022_Generate_Dream_Art.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
# Import the libraries needed to start putting Google's Deep Dream together
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from PIL import Image
import tensorflow as tf
import numpy as np
import imutils
import cv2

### Image Loader Function
Load the image, resize it to a width of 350. We then swap the color channels from BGR to RGB, and return the image as a NumPy array.

In [ ]:
def loadImage(imagePath):
    '''returns the image pixel array resiezd to a width of 350'''
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=350)
    # convert from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.array(image)
    return image

### Deprocess Function 
Utility function to convert a tensor into a valid image. It "undoes" the preprocessing done for Inception and then casts the pixel values to integers

In [ ]:
def deprocess(image):
    '''returns the deprocessed image'''
    image = 255 * (image + 1.0) 
    image /= 2.0
    image = tf.cast(image, tf.uint8)
    return image

### Calculate the loss after a single iteration of the DeepDream algorithm

We firstly add a batch dimension to the image and then grab the activations, from specified layers of the Inception network after performing a forward pass.

In [ ]:
def calculateLoss(image, model):
    '''returns the sum of losses'''
    image = tf.expand_dims(image, axis=0)
    layerActivations = model(image)

    # Create a list to store the intermediate losses
    losses = []

    # iterate over the layer activations
    for act in layerActivations:
        # compute the mean of each activation
        loss = tf.reduce_mean(act)
        # append these losses to the losses list
        losses.append(loss)

    # return the sum of the losses
    return tf.reduce_sum(losses)

## The Deep Dream function

This function performs a forward-pass and then taking the
gradients and using them to update the input image to generate the deep dream effects.

We input our model (inception network preferablly), the input image, the step size we use for our gradient updates and EPS, a tiny value used to prevent divide by zero errors.

In [ ]:
# This is new addition to TF2.0. When we use the **tf.function** decorator to a function.
# TensorFlow will automatically compile it into a graph which leads to faster execution.
@tf.function
def deepDreamSinglePass(model, image, stepSize, eps=1e-8):
    ''' returns a tuple of the loss and the updated image'''

    # Tells TenorFlow to record gradients
    with tf.GradientTape() as tape:
        # keep track of the image to calculate gradients and loss 
        tape.watch(image)
        loss = calculateLoss(image, model)

    # calculates the gradients of the loss with respect to the image
    gradients = tape.gradient(loss, image)
    # normalize the gradients 
    gradients /= tf.math.reduce_std(gradients) + eps 
    # K.maximum(K.mean(K.abs(grads)), K.epsilon())

    # adjusts the image with the normalized gradients 
    image = image + (gradients * stepSize)
    # clip its pixel values to the range [-1, 1]
    image = tf.clip_by_value(image, -1, 1)

    return (loss, image)

### Function that runs our Deep Dream Model

Up to this point we can perform a single forward-pass of the DeepDream algorithm. We now need to design our training loop that enables us to do this for multiple iterations.

The `fullDeepDreamModel` we will be making below will allow us to pass our model (inception) and input image for a specified number of iterations.

In [ ]:
def fullDeepDreamModel(model, image, iterations=50, stepSize=0.01):
    '''runs the deep dream algorithm for a specified number of iterations 
    and returns the depresocessed image'''

    # preprocess the image for the Inception network
    image = preprocess_input(image)

    # We iterate for the specified number of iterations
    for iteration in range(iterations):
        # use  our deepDreamSinglePass function to get the loss and the updated image
        (loss, image) = deepDreamSinglePass(model, image, stepSize)

        # print updates on the iteration progress
        if iteration % 50 == 0:
            print ("Iteration {} with loss {}".format(iteration, loss))

    return deprocess(image)

#### Define the layers we are going to use for the dream


In [ ]:
# Chose the layers you'll be using 
# other available layers: 
# mixed8, mixed9_0, mixed9_1, mixed10
# We try to get lower layer since they are resulting in more interesting art
names = ["mixed2", "mixed3","mixed4","mixed5", "mixed7"]

# define the octave scale and number of octaves (tweaking these values
# will produce different output dreams)
# OCTAVE_SCALE = 1.5 AND NUM_OCTAVES = 6 have generally given good results
# NUM_OCTAVES will determine number of iterations. Keep it small if doing a demo.
OCTAVE_SCALE = 1.5
NUM_OCTAVES = 1

### Execute Deep Dream



In [ ]:
# get our input image

sampleImage = loadImage('cchome-thanksgiving.jpg')
print("Loaded Image")

# load the pre-trained Inception model from disk
print("Loading Inception Model...")
baseModel = InceptionV3(include_top=False, weights="imagenet")
print("Loaded Inception Model")

In [ ]:
# Explore the layers you can use by looking at the model summary
# try using different mixed layers
baseModel.summary()

In [ ]:
# build our dreaming model
layers = [baseModel.get_layer(name).output for name in names]
deepDreamModel = tf.keras.Model(inputs=baseModel.input, outputs=layers)

# We convert the image to a TensorFlow constant for better performance,
image = tf.constant(sampleImage)

# We take the first two dimensions of the image and cast then them to float
baseShape = tf.cast(tf.shape(image)[:-1], tf.float32)

# Run our Deep Dream Generator!



In [ ]:
import matplotlib.pyplot as plt

# We iterate over the number of Octaves (sizes) we are going to create using Deep Dream
for n in range(NUM_OCTAVES):

    # get the width and height for the current octave and cast them to integers
    print("Working on octave {}".format(n))
    newShape = tf.cast(baseShape * (OCTAVE_SCALE ** n), tf.int32)

    # Resize the image with new shape and convert it to numpy before running 
    # through our DeepDream Model
    image = tf.image.resize(image, newShape).numpy()
    image = fullDeepDreamModel(model=deepDreamModel, image=image, iterations=1000, stepSize=0.0001)

# convert the final image to a numpy array and then save it to disk
finalImage = np.array(image)
cv2.imwrite('output.jpg', finalImage)

plt.figure(figsize=(20,10)) 
plt.imshow(finalImage)